## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [2]:
import datetime
import os

import pandas as pd
from dotenv import load_dotenv
from web3 import Web3
from web3.middleware import geth_poa_middleware

from notebooks import get_block_by_ts
from notebooks import get_twap_bpt_price
from notebooks.fees_and_bribs.constants import ARB_CORE_POOLS
from notebooks.fees_and_bribs.constants import AVAX_CORE_POOLS
from notebooks.fees_and_bribs.constants import BASE_CORE_POOLS
from notebooks.fees_and_bribs.constants import GNOSIS_CORE_POOLS
from notebooks.fees_and_bribs.constants import MAINNET_CORE_POOLS
from notebooks.fees_and_bribs.constants import POLYGON_CORE_POOLS

load_dotenv()
arb_web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))
eth_web3 = Web3(Web3.HTTPProvider(os.environ["ETHNODEURL"]))
base_web3 = Web3(Web3.HTTPProvider(os.environ["BASENODEURL"]))
gnosis_web3 = Web3(Web3.HTTPProvider(os.environ["GNOSISNODEURL"]))
avax_web3 = Web3(Web3.HTTPProvider(os.environ["AVALANCHENODEURL"]))
poly_web3 = Web3(Web3.HTTPProvider("https://polygon-rpc.com"))
poly_web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# ARBITRUM
# arb_block_now = arb_web3.eth.block_number - 1000
timestamp_now = 1694790000
arb_block = get_block_by_ts(timestamp_now, "arbitrum")  # 18 August 2023
# Given Arb block time, we want to look back 2 weeks:
arb_timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
arb_block_2_weeks_ago = get_block_by_ts(arb_timestamp_2_weeks_ago, "arbitrum")
# Convert to datetime:
arb_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
arb_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(arb_timestamp_2_weeks_ago)
# MAINNET
mainnet_block = get_block_by_ts(timestamp_now, "mainnet")
# Given mainnet block time, we want to look back 2 weeks:
mainnet_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
mainnet_block_2_weeks_ago = get_block_by_ts(mainnet_timestamp_2_weeks_ago, "mainnet")
# Convert to datetime:
mainnet_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
mainnet_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(mainnet_timestamp_2_weeks_ago)

# BASE
base_block = get_block_by_ts(timestamp_now, "base")
# Given base block time, we want to look back 2 weeks:
base_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
base_block_2_weeks_ago = get_block_by_ts(base_timestamp_2_weeks_ago, "base")
# Convert to datetime:
base_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
base_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(base_timestamp_2_weeks_ago)

# POLYGON
poly_block = get_block_by_ts(timestamp_now, "polygon")
# Given polygon block time, we want to look back 2 weeks:
poly_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
poly_block_2_weeks_ago = get_block_by_ts(poly_timestamp_2_weeks_ago, "polygon")
# Convert to datetime:
poly_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
poly_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(poly_timestamp_2_weeks_ago)

# GNOSIS
gnosis_block = get_block_by_ts(timestamp_now, "gnosis")
# Given gnosis block time, we want to look back 2 weeks:
gnosis_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
gnosis_block_2_weeks_ago = get_block_by_ts(gnosis_timestamp_2_weeks_ago, "gnosis")
# Convert to datetime:
gnosis_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
gnosis_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(gnosis_timestamp_2_weeks_ago)

# Avalanche
avax_block = get_block_by_ts(timestamp_now, "avalanche")
# Given avalanche block time, we want to look back 2 weeks:
avax_timestamp_2_weeks_ago = arb_timestamp_2_weeks_ago
avax_block_2_weeks_ago = get_block_by_ts(avax_timestamp_2_weeks_ago, "avalanche")
# Convert to datetime:
avax_datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
avax_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(avax_timestamp_2_weeks_ago)

print("Collecting bpt prices for Gnosis")
gnosis_bpt_twap_prices = {}
for gnosis_pool in GNOSIS_CORE_POOLS:
    gnosis_bpt_twap_prices[gnosis_pool] = get_twap_bpt_price(gnosis_pool, 'gnosis', gnosis_web3,
                                                             start_date=gnosis_datetime_now,
                                                             block_number=gnosis_block)

print("Collecting bpt prices for Avalanche")
avax_bpt_twap_prices = {}
for avax_pool in AVAX_CORE_POOLS:
    avax_bpt_twap_prices[avax_pool] = get_twap_bpt_price(avax_pool, 'avalanche', avax_web3,
                                                         start_date=avax_datetime_now, block_number=avax_block)
print("Collecting bpt prices for Base")
base_bpt_twap_prices = {}
for base_pool in BASE_CORE_POOLS:
    base_bpt_twap_prices[base_pool] = get_twap_bpt_price(base_pool, 'base', base_web3,
                                                         start_date=base_datetime_now, block_number=base_block)
print("Collecting bpt prices for Arbitrum")
arb_bpt_twap_prices = {}
for arb_pool in ARB_CORE_POOLS:
    arb_bpt_twap_prices[arb_pool] = get_twap_bpt_price(arb_pool, 'arbitrum', arb_web3,
                                                       start_date=arb_datetime_now,
                                                       block_number=arb_block)

print("Collecting bpt prices for Mainnet")
mainnet_bpt_twap_prices = {}
for mainnet_pool in MAINNET_CORE_POOLS:
    mainnet_bpt_twap_prices[mainnet_pool] = get_twap_bpt_price(mainnet_pool, 'mainnet', eth_web3,
                                                               start_date=mainnet_datetime_now,
                                                               block_number=mainnet_block)
print("Collecting bpt prices for Polygon")
polygon_bpt_twap_prices = {}
for polygon_pool in POLYGON_CORE_POOLS:
    polygon_bpt_twap_prices[polygon_pool] = get_twap_bpt_price(polygon_pool, 'polygon', poly_web3,
                                                               start_date=poly_datetime_now,
                                                               block_number=poly_block)

# Convert to dataframe and print, merge all three dataframes
arb_bpt_twap_prices_df = pd.DataFrame.from_dict(arb_bpt_twap_prices, orient='index')
mainnet_bpt_twap_prices_df = pd.DataFrame.from_dict(mainnet_bpt_twap_prices, orient='index')
polygon_bpt_twap_prices_df = pd.DataFrame.from_dict(polygon_bpt_twap_prices, orient='index')
base_bpt_twap_prices_df = pd.DataFrame.from_dict(base_bpt_twap_prices, orient='index')
gnosis_bpt_twap_prices_df = pd.DataFrame.from_dict(gnosis_bpt_twap_prices, orient='index')
avax_bpt_twap_prices_df = pd.DataFrame.from_dict(avax_bpt_twap_prices, orient='index')

## Fetching data from the Balancer subgraphs

In [4]:
from notebooks import get_balancer_pool_snapshots
from notebooks.fees_and_bribs.constants import BASE_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import POLYGON_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import ARB_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import MAINNET_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import GNOSIS_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import AVALANCHE_BALANCER_GRAPH_URL
from typing import Dict

arbi_pool_snapshots_now = get_balancer_pool_snapshots(arb_block, ARB_BALANCER_GRAPH_URL)
arbi_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(arb_block_2_weeks_ago), ARB_BALANCER_GRAPH_URL)

mainnet_pool_snapshots_now = get_balancer_pool_snapshots(mainnet_block, MAINNET_BALANCER_GRAPH_URL)
mainnet_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(mainnet_block_2_weeks_ago),
                                                                 MAINNET_BALANCER_GRAPH_URL)

polygon_pool_snapshots_now = get_balancer_pool_snapshots(poly_block, POLYGON_BALANCER_GRAPH_URL)
polygon_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(poly_block_2_weeks_ago),
                                                                 POLYGON_BALANCER_GRAPH_URL)

base_pool_snapshots_now = get_balancer_pool_snapshots(base_block, BASE_BALANCER_GRAPH_URL)
base_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(base_block_2_weeks_ago),
                                                              BASE_BALANCER_GRAPH_URL)

gnosis_pool_snapshots_now = get_balancer_pool_snapshots(gnosis_block, GNOSIS_BALANCER_GRAPH_URL)
gnosis_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(gnosis_block_2_weeks_ago),
                                                                GNOSIS_BALANCER_GRAPH_URL)

avax_pool_snapshots_now = get_balancer_pool_snapshots(avax_block, AVALANCHE_BALANCER_GRAPH_URL)
avax_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(avax_block_2_weeks_ago),
                                                              AVALANCHE_BALANCER_GRAPH_URL)

## Extract fee data for CORE pools:


In [7]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: str, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.datetime) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Calculate fees
        pool_fee = float(pool_snapshot_now['protocolFee']) - float(pool_snapshot_2_weeks_ago['protocolFee'])
        pool_swap_fee = float(pool_snapshot_now['swapFees']) - float(pool_snapshot_2_weeks_ago['swapFees'])
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = arb_bpt_twap_prices[pool] if chain == 'arbitrum' else mainnet_bpt_twap_prices[
            pool] if chain == 'mainnet' else polygon_bpt_twap_prices[pool] if chain == 'polygon' else \
            base_bpt_twap_prices[pool] if chain == 'base' else gnosis_bpt_twap_prices[pool] if chain == 'gnosis' else \
                avax_bpt_twap_prices[pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['symbol']].append({
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': arb_datetime_2_weeks_ago,
                'time_to': arb_datetime_now,
                'chain': chain,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain, start_date) or 0
                token_fees[pool_snapshot_now['pool']['symbol']].append({
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': arb_datetime_2_weeks_ago,
                    'time_to': arb_datetime_now,
                    'chain': chain,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['symbol']] = {
            'pool_fee': round(pool_fee, 2),
            'swap_fee': round(pool_swap_fee, 2),
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                            token_fees[
                                                pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': arb_datetime_2_weeks_ago,
            'time_to': arb_datetime_now,
            'chain': chain,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']],
            'pool_addr': pool_snapshot_now['pool']['address'],
        }
    return fees


arb_fees = collect_fee_info(ARB_CORE_POOLS, 'arbitrum', arbi_pool_snapshots_now,
                            arbi_pool_snapshots_2_weeks_ago, arb_datetime_now)
mainnet_fees = collect_fee_info(MAINNET_CORE_POOLS, 'mainnet', mainnet_pool_snapshots_now,
                                mainnet_pool_snapshots_2_weeks_ago, mainnet_datetime_now)
polygon_fees = collect_fee_info(POLYGON_CORE_POOLS, 'polygon', polygon_pool_snapshots_now,
                                polygon_pool_snapshots_2_weeks_ago, poly_datetime_now)
base_fees = collect_fee_info(BASE_CORE_POOLS, 'base', base_pool_snapshots_now, base_pool_snapshots_2_weeks_ago,
                             base_datetime_now)
gnosis_fees = collect_fee_info(GNOSIS_CORE_POOLS, 'gnosis', gnosis_pool_snapshots_now,
                               gnosis_pool_snapshots_2_weeks_ago, gnosis_datetime_now)
avax_fees = collect_fee_info(AVAX_CORE_POOLS, 'avalanche', avax_pool_snapshots_now,
                             avax_pool_snapshots_2_weeks_ago, avax_datetime_now)
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**arb_fees, **mainnet_fees, **polygon_fees, **base_fees, **gnosis_fees, **avax_fees}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [8]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain', 'pool_fee'], ascending=False)

,pool_fee,swap_fee,bpt_token_fee,bpt_token_fee_in_usd,token_fees_in_usd,time_from,time_to,chain,pool_addr
ECLP-WMATIC-stMATIC,1584.96,249.39,2946.55,1591.32,0,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon,0xf0ad209e2e969eaaa8c882aac71f02d8a047d5c2
maticX-WMATIC-BPT,1450.10,326.70,2724.34,1459.21,0,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon,0xcd78a20c597e367a4e478a2411ceb790604d7c8f
ECLP-WMATIC-MATICX,403.27,71.47,755.16,406.22,0,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon,0xee278d943584dd8640eaf4cc6c7a5c80c0073e85
stMATIC-WMATIC-BPT,2.58,1.34,4.62,2.47,0,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon,0x89b753153678bc434c610b7e9182297cada8ff29
wstETH-WETH-BPT,0.00,0.00,0.00,0.00,0.00,2023-09-01 18:00:00,2023-09-15 18:00:00,polygon,0xdc31233e09f3bf5bfe5c10da2014677c23b6894c
B-rETH-STABLE,15892.61,2300.19,0.00,0.00,15797.35,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet,0x1e19cf2d73a72ef1332c882f20534b6519be0276
50rETH-50BADGER,9939.15,2997.35,166.20,9826.84,0,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet,0x1ee442b5326009bb18f2f472d3e0061513d1a0ff
swETH-WETH-BPT,5502.29,634.17,3.43,5510.58,0,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet,0xe7e2c68d3b13d905bbb636709cf4dfd21076b9d2
ankrETH/wstETH,3867.44,126.65,2.32,3789.32,0,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet,0xdfe6e7e18f6cc65fa13c8d8966013d4fda74b6ba
20WETH-80ALCX,2393.07,4118.43,83.27,2069.88,0,2023-09-01 18:00:00,2023-09-15 18:00:00,mainnet,0xf16aee6a71af1a9bc8f56975a4c2705ca7a782bc


## Now let's calculate bribes paid to the pools

In [46]:
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(eth_web3, mainnet_block)

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.
FEE = Decimal(0.5)  # 50% goes to fees


# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str, fees_to_distribute: Decimal) -> Dict[str, Dict]:
    pool_bribs = {}
    # Calculate pool share in fees
    total_fees = sum([data['pool_fee'] for pool, data in fees.items()])
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        pool_share = pool_fees / Decimal(total_fees)
        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "chain": chain,
            "aura_bribes": round(pool_share * fees_to_distribute * aura_vebal_share * FEE, 2),
            "bal_bribes": round(pool_share * fees_to_distribute * (1 - aura_vebal_share) * FEE, 2),
            "pool_fees_collected": pool_fees,
            "revenue_fees": pool_fees * FEE,
            # "pool_share": f"{round(pool_share * 100, 2)}%",
            "pool_addr": data['pool_addr'],
            "pool_name": pool,
        }
    return pool_bribs


# TODO: Move to constants
mainnet_bribes = calc_and_split_bribes(mainnet_fees, 'mainnet', Decimal(81763))
arb_bribes = calc_and_split_bribes(arb_fees, 'arbitrum', Decimal(18523))
polygon_bribes = calc_and_split_bribes(polygon_fees, 'polygon', Decimal(5162.59))
base_bribes = calc_and_split_bribes(base_fees, 'base', Decimal(67118))
gnosis_bribes = calc_and_split_bribes(gnosis_fees, 'gnosis', Decimal(597))
avax_bribes = calc_and_split_bribes(avax_fees, 'avalanche', Decimal(10297))
# Convert to dataframe
joint_bribes_data = {**arb_bribes, **mainnet_bribes, **polygon_bribes, **base_bribes, **gnosis_bribes, **avax_bribes}
# Sort by chain:
joint_bribes_data = {k: v for k, v in sorted(joint_bribes_data.items(), key=lambda item: item[1]['chain'])}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
# Dump into csv and prefix with dates
joint_bribes_df.to_csv(f'./output/bribes_{arb_datetime_2_weeks_ago.date()}_{arb_datetime_now.date()}.csv')
joint_bribes_df

,chain,aura_bribes,bal_bribes,pool_fees_collected,revenue_fees,pool_addr,pool_name
wstETH/rETH/cbETH,arbitrum,461.01,698.09,1981.51,990.755,0x4a2f6ae7f3e5d715689530873ec35593dc28951b,wstETH/rETH/cbETH
4POOL-BPT,arbitrum,110.69,167.62,475.79,237.895,0x423a1323c871abc9d89eb06855bf5347048fc4a5,4POOL-BPT
RDNT-WETH,arbitrum,2643.32,4002.69,11361.56,5680.780,0x32df62dc3aed2cd6224193052ce665dc18165841,RDNT-WETH
wstETH/rETH/sfrxETH,arbitrum,0.00,0.01,0.02,0.010,0x0c8972437a38b389ec83d1e666b69b8a4fcf8bfd,wstETH/rETH/sfrxETH
sAVAX-WAVAX-BPT,avalanche,1021.00,1546.07,2462.77,1231.385,0xfd2620c9cfcec7d152467633b3b0ca338d3d78cc,sAVAX-WAVAX-BPT
ggAVAX-WAVAX-BPT,avalanche,173.67,262.98,418.90,209.450,0xc13546b97b9b1b15372368dc06529d7191081f5b,ggAVAX-WAVAX-BPT
yyAVAX-WAVAX-BPT,avalanche,328.18,496.95,791.60,395.800,0x9fa6ab3d78984a69e712730a2227f20bcc8b5ad9,yyAVAX-WAVAX-BPT
USDC-USDT-BPT,avalanche,50.75,76.85,122.41,61.205,0xb26f0e66317846bd5fe0cbaa1d269f0efeb05c96,USDC-USDT-BPT
BPT,avalanche,0.00,0.00,0.00,0.000,0x55bec22f8f6c69137ceaf284d9b441db1b9bfedc,BPT
rETH-WETH-BPT,base,0.00,0.00,0.00,0.000,0xc771c1a5905420daec317b154eb13e4198ba97d0,rETH-WETH-BPT


In [47]:
from notebooks import fetch_all_pools_info

mainnet_pools_info = fetch_all_pools_info('mainnet')
arb_pools_info = fetch_all_pools_info('arbitrum')
polygon_pools_info = fetch_all_pools_info('polygon')
base_pools_info = fetch_all_pools_info('base')
gnosis_pools_info = fetch_all_pools_info('gnosis')
avax_pools_info = fetch_all_pools_info('avalanche')

mainnet_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
                  mainnet_pools_info}
arb_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
              arb_pools_info}
polygon_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
                  polygon_pools_info}
base_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
               base_pools_info}
gnosis_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
                 gnosis_pools_info}
avax_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
               avax_pools_info}
# Put it all into mapping per chain:
mapped_gauges = {
    'mainnet': mainnet_gauges,
    'arbitrum': arb_gauges,
    'polygon': polygon_gauges,
    'base': base_gauges,
    'gnosis': gnosis_gauges,
    'avalanche': avax_gauges,
}

In [76]:
# Load csv sheet to compare with the data collected above:
bribes_df = pd.read_csv('compare_sheet.csv')
# To dict:
bribes_to_compare = bribes_df.to_dict(orient='records')
# Each dict in list needs target to be Web3.toChecksumAddress
bribes_csv_checksummed = []
for bribe in bribes_to_compare:
    bribes_csv_checksummed.append({
        'target': Web3.to_checksum_address(bribe['target']),
        'platform': bribe['platform'],
        'amount': bribe['amount'],
    })
# These are bribes paid de-facto, so we need to convert them to the same format as we have in our data
bribes_final_csv = {}
for bribe in bribes_csv_checksummed:
    if bribe['target'] not in bribes_final_csv:
        bribes_final_csv[bribe['target']] = {}
    bribes_final_csv[bribe['target']][bribe['platform']] = Decimal(bribe['amount'])

## Compare data from the sheet with the data we collected
What does table below represent:
- aura_automated_bribe: bribes paid to aura per pool calculated in this notebook
- aura_actual_bribe: actual bribes paid to aura per pool from the sheet
- aura_bribe_delta: difference between aura_automated_bribe and aura_actual_bribe
- aura_bribe_delta_%: difference between aura_automated_bribe and aura_actual_bribe in %

Same thing for balancer side:
- bal_automated_bribe: bribes paid to balancer per pool calculated in this notebook
- bal_actual_bribe: actual bribes paid to balancer per pool from the sheet
- bal_bribe_delta: difference between bal_automated_bribe and bal_actual_bribe
- bal_bribe_delta_%: difference between bal_automated_bribe and bal_actual_bribe in %

In [78]:
# Now we need to compare the data we collected with the data from the sheet
# We need to compare bribes paid to each pool
joint_bribes_dict = joint_bribes_df.to_dict(orient='records')
bribes_delta = {}
for gauge, data in bribes_final_csv.items():
    for automated_bribe in joint_bribes_dict:
        filtered_gauges = mapped_gauges[automated_bribe['chain']]
        # If this condition is true, we can compare the bribes
        if filtered_gauges[automated_bribe['pool_addr']] == gauge:
            # Calculate delta between automated bribes and bribes from the sheet
            # Calculate delta between automated_bribe['aura_bribes'] and data['aura'] in %
            if automated_bribe['aura_bribes'] > data['aura']:
                aura_bribe_delta = automated_bribe['aura_bribes'] - data['aura']
                aura_bribe_delta_pct = (automated_bribe['aura_bribes'] - data['aura']) / data['aura'] * 100 if \
                    data['aura'] != 0 else 0
            else:
                aura_bribe_delta = data['aura'] - automated_bribe['aura_bribes']
                aura_bribe_delta_pct = (data['aura'] - automated_bribe['aura_bribes']) / data['aura'] * 100 if \
                    data['aura'] != 0 else 0
            if automated_bribe['bal_bribes'] > data['balancer']:
                bal_bribe_delta = automated_bribe['bal_bribes'] - data['balancer']
                bal_bribe_delta_pct = (automated_bribe['bal_bribes'] - data['balancer']) / data['balancer'] * 100 if \
                    data['balancer'] != 0 else 0
            else:
                bal_bribe_delta = data['balancer'] - automated_bribe['bal_bribes']
                bal_bribe_delta_pct = (data['balancer'] - automated_bribe['bal_bribes']) / data['balancer'] * 100 if \
                    data['balancer'] != 0 else 0
                
            bribes_delta[automated_bribe['pool_name']] = {
                'gauge': gauge,
                'chain': automated_bribe['chain'],
                'aura_automated_bribe': automated_bribe['aura_bribes'],
                'aura_actual_bribe': round(data['aura'], 2),
                'aura_bribe_delta': round(aura_bribe_delta, 2),
                'aura_bribe_delta_%': f'{round(aura_bribe_delta_pct, 2)}%',
                'bal_automated_bribe': automated_bribe['bal_bribes'],
                'bal_actual_bribe': round(data['balancer'], 2),
                'bal_bribe_delta': round(bal_bribe_delta, 2),
                'bal_bribe_delta_%': f'{round(bal_bribe_delta_pct, 2)}%',
            }
# To df and print out without index:
bribes_delta_df = pd.DataFrame.from_dict(bribes_delta, orient='index')
bribes_delta_df.sort_values(by=['chain', 'aura_bribe_delta'], ascending=False)
bribes_delta_df

,gauge,chain,aura_automated_bribe,aura_actual_bribe,aura_bribe_delta,aura_bribe_delta_%,bal_automated_bribe,bal_actual_bribe,bal_bribe_delta,bal_bribe_delta_%
B-rETH-STABLE,0x79eF6103A513951a3b25743DB509E267685726B7,mainnet,5074.67,2944.03,2130.64,72.37%,7684.40,5467.48,2216.92,40.55%
swETH-WETH-BPT,0xee01c0d9c0439c94D314a6ecAE0490989750746C,mainnet,1770.19,4953.73,3183.54,64.27%,2680.55,0.00,2680.55,0%
ECLP-wstETH-cbETH,0x4D4264aebf65Bb1727bb5438E0b2Aaf86186DA50,mainnet,0.00,0.00,0.00,0%,0.00,0.00,0.00,0%
20WETH-80ALCX,0x183D73dA7adC5011EC3C46e33BB50271e59EC976,mainnet,664.92,0.00,664.92,0%,1006.86,0.00,1006.86,0%
B-staFiETH-WETH-Stable,0x10a361766e64D7983a97202ac3a0F4cee06Eb717,mainnet,389.46,0.00,389.46,0%,589.74,0.00,589.74,0%
GHO/USDT/USDC,0xf720e9137baa9C7612e6CA59149a5057ab320cFa,mainnet,0.00,0.00,0.00,0%,0.00,0.00,0.00,0%
ankrETH/wstETH,0xbf65b3fA6c208762eD74e82d4AEfCDDfd0323648,mainnet,1217.26,0.00,1217.26,0%,1843.26,0.00,1843.26,0%
50wstETH-50LDO,0x95201B61EF19C867dA0D093DF20021e1a559452c,mainnet,197.05,0.00,197.05,0%,298.38,0.00,298.38,0%
50rETH-50BADGER,0x87012b0C3257423fD74a5986F81a0f1954C17a1d,mainnet,3156.73,1793.05,1363.68,76.05%,4780.13,3329.95,1450.18,43.55%
50rETH-50RPL,0x2C2179abce3413E27BDA6917f60ae37F96D01826,mainnet,405.02,0.00,405.02,0%,613.31,0.00,613.31,0%
